<a href="https://colab.research.google.com/github/limkaram/Natural_language_processing_with_deep_learning/blob/main/20newsgroups_Classification_with_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MLP 활용 20개 뉴스 그룹 분류 실습

1. 데이터 이해 및 전처리

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups  # 20개의 다른 주제를 지닌 18,846개의 뉴스 그룹 이메일 데이터
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [ ]:
newsdata = fetch_20newsgroups(subset = 'train')  # train 데이터만 반환

In [ ]:
print(newsdata.keys())

In [ ]:
data = pd.DataFrame(newsdata.data, columns=['email'])
data['target'] = pd.Series(newsdata.target)
data.info()

In [ ]:
import seaborn as sns

# target 개수 분포 확인
sns.countplot(x=data['target'])
data.groupby(by='target').size()

In [ ]:
# 훈련 데이터, 테스트 데이터 본문, 레이블 분류
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True)

train_email = data['email']
train_label = data['target']

test_email = newsdata_test.data
test_label = newsdata_test.target

In [ ]:
# preprocessing 진행
max_words = 10000  # 빈도수 상위 10000개의 단어만 사용
num_classes = 20  # 레이블 수

In [ ]:
def prepare_data(train_data, test_data, mode):
    t = Tokenizer(num_words=max_words)
    t.fit_on_texts(train_data)
    X_train = t.texts_to_matrix(train_data, mode=mode)  # 샘플 수 x max_words 크기 행렬 생성
    X_test = t.texts_to_matrix(test_data, mode=mode)  # 샘플 수 x max_words 크기 행렬 생성
    return X_train, X_test, t.index_word

* texts_to_matrix(mode='mode 인자') mode 가능 인자
    - 'count' : DTM에서 단어 갯수로 구성
    - 'binary' : DTM에서 단어 존재시 1, 미존재시 0으로 적용
    - 'tfidf' : tfidf 실수 값으로 적용
    - 'freq' : 각 문서에서 각 단어의 등장 횟수 / 각 문서의 크기(각 문서에서 등장한 모든 단어의 개수의 총 합)으로 표현

In [ ]:
X_train, X_test, index_to_word = prepare_data(train_email, test_email, 'binary')
y_train = to_categorical(train_label, num_classes)  # 원-핫 인코딩
y_test = to_categorical(test_label, num_classes)  # 원-핫 인코딩

In [ ]:
print('훈련 샘플 본문의 크기 : {}'.format(X_train.shape))
print('훈련 샘플 레이블의 크기 : {}'.format(y_train.shape))
print('테스트 샘플 본문의 크기 : {}'.format(X_test.shape))
print('테스트 샘플 레이블의 크기 : {}'.format(y_test.shape))

2. MLP 모델 설계

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def fit_and_evaluate(X_train, y_train, X_test, y_test):
    model = Sequential()
    model.add(Dense(256, input_shape=(max_words, ), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1, validation_split=0.1)
    score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)
    return score[1]

* Dropout(0.5) : overfiting을 막기 위해 각 은닉층의 뉴런의 반을 가리고 학습

In [26]:
modes = ['binary', 'count', 'tfidf', 'freq']  # texts_to_matrix()의 4개 모드 전부 학습 예정
for mode in modes:
    X_train, X_test, _ = prepare_data(train_email, test_email, mode)
    score = fit_and_evaluate(X_train, y_train, X_test, y_test)  # 모델을 훈련하고 평가
    print(mode + ' 모드 테스트 정확도 : ', score)

Epoch 1/5
80/80 [==============================] - 1s 7ms/step - loss: 2.2625 - accuracy: 0.3418 - val_loss: 0.9526 - val_accuracy: 0.8295
Epoch 2/5
80/80 [==============================] - 0s 6ms/step - loss: 0.8593 - accuracy: 0.7672 - val_loss: 0.4569 - val_accuracy: 0.8860
Epoch 3/5
80/80 [==============================] - 0s 6ms/step - loss: 0.4331 - accuracy: 0.8852 - val_loss: 0.3494 - val_accuracy: 0.9117
Epoch 4/5
80/80 [==============================] - 0s 6ms/step - loss: 0.2576 - accuracy: 0.9354 - val_loss: 0.3021 - val_accuracy: 0.9249
Epoch 5/5
80/80 [==============================] - 0s 6ms/step - loss: 0.1750 - accuracy: 0.9594 - val_loss: 0.2773 - val_accuracy: 0.9258
binary모드 테스트 정확도 :  0.8321827054023743
Epoch 1/5
80/80 [==============================] - 1s 8ms/step - loss: 2.7244 - accuracy: 0.2397 - val_loss: 1.5827 - val_accuracy: 0.7491
Epoch 2/5
80/80 [==============================] - 1s 6ms/step - loss: 1.4034 - accuracy: 0.6399 - val_loss: 0.6925 - val_accur